## Well log availability check


In [1]:
import glob
import numpy as np
import lasio
import pandas as pd
import json
from pandas import merge_ordered

In [2]:
#use glob.glob to check the available well in our folder
for filename in glob.iglob(r'C:\Users\Henky\JUPYTER\logChecker\*.las', recursive=True):
    print (filename)

C:\Users\Henky\JUPYTER\logChecker\well-1.las
C:\Users\Henky\JUPYTER\logChecker\well-2.las
C:\Users\Henky\JUPYTER\logChecker\well-3.las


In [3]:
#simply add ** to the root directory to also access the subfolder under specified directory
for filename in glob.iglob(r'C:\Users\Henky\JUPYTER\logChecker\**\*.las', recursive=True):
    data=lasio.read(filename) #load data las using lasio
    well_data=data.df()
    print (filename, well_data.columns)

C:\Users\Henky\JUPYTER\logChecker\well-1.las Index(['CALI', 'DT', 'GR', 'ILD', 'MSFL', 'NPHI_LS', 'RHOB'], dtype='object')
C:\Users\Henky\JUPYTER\logChecker\well-2.las Index(['CALI', 'DT', 'GR', 'ILD', 'MSFL', 'NPHI_LS', 'RHOB'], dtype='object')
C:\Users\Henky\JUPYTER\logChecker\well-3.las Index(['CALI', 'DT', 'GR', 'ILD', 'MSFL', 'NPHI_LS', 'RHOB'], dtype='object')
C:\Users\Henky\JUPYTER\logChecker\new\another-well.las Index(['FORCE_2020_LITHOFACIES_CONFIDENCE', 'FORCE_2020_LITHOFACIES_LITHOLOGY',
       'CALI', 'BS', 'DCAL', 'MUDWEIGHT', 'ROP', 'RDEP', 'RSHA', 'RMED', 'RXO',
       'SP', 'DTCO', 'NPHI', 'PEF', 'GR', 'RHOB', 'DRHO', 'DEPTH_MD', 'X_LOC',
       'Y_LOC', 'Z_LOC'],
      dtype='object')
C:\Users\Henky\JUPYTER\logChecker\new\puk1.las Index(['CALI', 'DT', 'GR', 'ILD', 'MSFL', 'NPHI_LS', 'RHOB'], dtype='object')
C:\Users\Henky\JUPYTER\logChecker\new\well-try.las Index(['CALI', 'UNKNOWN:1', 'DEPTH', 'DRHO', 'DT', 'GR', 'ILD', 'UNKNOWN:2',
       'MSFL', 'NPHI', 'UNKNOWN:3', 

### Now we have the report of the logs availability on our LAS files, let's improve the report format
I wrote some functions to produce a better report format, we will use this functions to shorten our code writing.

In [4]:
def load_data(filename):
    l = lasio.read(filename)
    data_well = l.df()
    log = list(data_well.columns.values)
    wellname=filename.split('\\')
    wellname=wellname[-1]
    header = [{
        'Well Name': wellname,
        'START':l.well.STRT.value,
        'STOP':l.well.STOP.value,
        'STEP':l.well.STEP.value,
        'Location': filename
    }]
    data_well['WELL'] = filename  
    log_list = pd.DataFrame(header)
    return data_well, log_list

def merge_alias(db, alias, logs_selected):
    well = db['WELL'].unique()
    merged_data = pd.DataFrame()
    for i in range(len(well)):
        data = db.where(db['WELL']==well[i]).dropna(axis=1, how='all')
        for j in range(len(alias)):
            welllog_name = list(set(data.columns).intersection(alias.get(list(alias)[j])))
            samelog = data[welllog_name]
            count_log = dict(sorted(zip(welllog_name, samelog.count()), key=lambda item: item[1], reverse=True))
            welllog_name = list(count_log.keys())
            if (len(welllog_name)!=0):
                #If more than one log aliases exist, normalize each log to have same data range in the same depth
                if (len(welllog_name)>1):
                    alias_logs = data[welllog_name].dropna()
                    if (len(alias_logs)!=0):
                        a = []; b = []; c = []
                        for n in range(len(alias_logs.columns)):
                            q1 = alias_logs[welllog_name[n]].quantile(0.1)
                            q9 = alias_logs[welllog_name[n]].quantile(0.9)
                            a.append(q1)
                            b.append(q9)
                            c = [b-a for (a,b) in zip(a,b)]
                            c = list(map(lambda x: x/c[0],c))
                        for n in range(len(welllog_name)):
                            data.loc[:, welllog_name[n]] *= 1/c[n]
                    for k in range(len(welllog_name)-1):
                        data[welllog_name[0]].fillna(data[welllog_name[k+1]], inplace=True)
                data[list(alias)[j]] = data[welllog_name[0]]   
        merged_data = merged_data.append(data)
        merged_data = merged_data[merged_data['WELL'].notna()]
        new_list=[]
        for x in logs_selected:
            if x in list(merged_data.columns):
                new_list.append(x)
        merged_data = merged_data[new_list]
    return merged_data, data

def tablecheck(filename, logs_selected):
    data_well, log_list=load_data(filename)
    merged_data, data=merge_alias(data_well, alias, logs_selected)
    df=pd.DataFrame([["X"]*(len(list(merged_data.columns)))], columns=list(merged_data.columns))
    df2=pd.concat([df, log_list], axis=1)
    return df2, log_list


    

## re-do the use of glob library and use the functions created.

In [5]:
#I also put a library (JSON) to help us merge the log name and avoid the inconsistancy of the result

with open('alias.json') as file:
    alias = json.load(file) #load the JSON File
    
#define the logs that we are interested in our las files, check the name on the JSON library
logs_selected= ['CAL', 'RXO', 'GR', 'POR', 'DRES', 'DT', 'DENS', 'DRHO'] 

#create an empty dataframe as the report container, use the glob library to explore folders, print the report
df=pd.DataFrame(columns=logs_selected)
for filename in glob.iglob(r'C:\Users\Henky\JUPYTER\logChecker\**\*.las', recursive=True):
    df2, log_list=tablecheck(filename, logs_selected)
    df=df.append(df2, ignore_index=True, sort=False)
    logCheckReport=df #create the report from the df
    
#print the result    
print (logCheckReport)

  CAL RXO GR POR DRES   DT DENS DRHO         Well Name      START  \
0   X   X  X   X    X    X    X  NaN        well-1.las  3001.0000   
1   X   X  X   X    X    X    X  NaN        well-2.las  3001.0000   
2   X   X  X   X    X    X    X  NaN        well-3.las  3001.0000   
3   X   X  X   X  NaN  NaN    X    X  another-well.las    25.0000   
4   X   X  X   X    X    X    X  NaN          puk1.las  1860.0000   
5   X   X  X   X    X    X    X    X      well-try.las   506.2728   

          STOP   STEP                                           Location  
0  4460.000000  0.500       C:\Users\Henky\JUPYTER\logChecker\well-1.las  
1  4521.500000  0.500       C:\Users\Henky\JUPYTER\logChecker\well-2.las  
2  4759.500000  0.500       C:\Users\Henky\JUPYTER\logChecker\well-3.las  
3  3210.005615  0.152  C:\Users\Henky\JUPYTER\logChecker\new\another-...  
4  4918.000000  0.500     C:\Users\Henky\JUPYTER\logChecker\new\puk1.las  
5  1452.981600  0.000  C:\Users\Henky\JUPYTER\logChecker\new\well-

In [6]:
#rearrange the order of the columns
header=(list(log_list.columns)+logs_selected)
logCheckReport=logCheckReport[header]
print (logCheckReport)

          Well Name      START         STOP   STEP  \
0        well-1.las  3001.0000  4460.000000  0.500   
1        well-2.las  3001.0000  4521.500000  0.500   
2        well-3.las  3001.0000  4759.500000  0.500   
3  another-well.las    25.0000  3210.005615  0.152   
4          puk1.las  1860.0000  4918.000000  0.500   
5      well-try.las   506.2728  1452.981600  0.000   

                                            Location CAL RXO GR POR DRES   DT  \
0       C:\Users\Henky\JUPYTER\logChecker\well-1.las   X   X  X   X    X    X   
1       C:\Users\Henky\JUPYTER\logChecker\well-2.las   X   X  X   X    X    X   
2       C:\Users\Henky\JUPYTER\logChecker\well-3.las   X   X  X   X    X    X   
3  C:\Users\Henky\JUPYTER\logChecker\new\another-...   X   X  X   X  NaN  NaN   
4     C:\Users\Henky\JUPYTER\logChecker\new\puk1.las   X   X  X   X    X    X   
5  C:\Users\Henky\JUPYTER\logChecker\new\well-try...   X   X  X   X    X    X   

  DENS DRHO  
0    X  NaN  
1    X  NaN  
2    X  NaN

In [7]:
#export the result as excel
logCheckReport.to_excel("log checklist.xlsx")  